# Runtime (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/5.0.0/lib/net5.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
open Common

In [ ]:
//// test

open FileSystem

## isWindows

In [ ]:
let inline isWindows () =
    System.Runtime.InteropServices.RuntimeInformation.IsOSPlatform System.Runtime.InteropServices.OSPlatform.Windows

In [ ]:
//// test

isWindows ()

True

## splitCommand

In [ ]:
type private CommandParseStep =
    | Start
    | Path of quoted: bool
    | Arguments

let splitCommand (command: string) =
    let rec loop (path, args) chars step =
        match chars, step with
        | ('"' | '\'') :: tail, _ when path = "" -> loop (path, args) tail (Path true)
        | ('"' | '\'') :: tail, Path true -> loop (path, args) tail (Path false)
        | ' ' :: tail, Path true -> loop ($"{path} ", args) tail (Path true)
        | ' ' :: tail, (Start | Path _) -> loop (path, args) tail Arguments
        | char :: tail, Arguments -> loop (path, $"{args}{char}") tail Arguments
        | char :: tail, _ -> loop ($"{path}{char}", args) tail step
        | _, _ -> path |> String.replace @"\" "/", args
    let path, args = loop ("", "") (command |> Seq.toList) Start
    let workingDirectory, fileName =
        if path |> String.startsWith "./" || path |> String.contains "/"
        then path |> System.IO.Path.GetDirectoryName |> String.replace @"\" "/", System.IO.Path.GetFileName path
        else ".", path
    workingDirectory, fileName, args

In [ ]:
//// test

splitCommand ""
|> _equal (".", "", "")

splitCommand "/a/b/c"
|> _equal ("/a/b", "c", "")

splitCommand "cat file.txt"
|> _equal (".", "cat", "file.txt")

splitCommand @"..\..\file.exe file1.txt file2.txt"
|> _equal ("../..", "file.exe", "file1.txt file2.txt")

splitCommand @"c:\dir\file.exe ""file1.txt file2.txt"""
|> _equal (@"c:/dir", "file.exe", @"""file1.txt file2.txt""")

splitCommand @"""..\..\dir name\file.exe"" ""file 1.txt"" file2.txt"
|> _equal ("../../dir name", "file.exe", @"""file 1.txt"" file2.txt")

splitCommand @"""..\..\file 1.exe"" -c \\""echo 1\\"""
|> _equal ("../..", "file 1.exe", @"-c \\""echo 1\\""")

splitCommand @"..\..\file 1.exe -c \\""echo 1\\"""
|> _equal ("../..", "file", @"1.exe -c \\""echo 1\\""")

( ., ,  )
( /a/b, c,  )
( ., cat, file.txt )
( ../.., file.exe, file1.txt file2.txt )
( c:/dir, file.exe, "file1.txt file2.txt" )
( ../../dir name, file.exe, "file 1.txt" file2.txt )
( ../.., file 1.exe, -c \\"echo 1\\" )
( ../.., file, 1.exe -c \\"echo 1\\" )


## executeAsync

In [ ]:
type ExecutionLine =
    {
        ProcessId : int
        Line : string
        Error : bool
    }

type ExecutionOptions =
    {
        Command : string
        CancellationToken : System.Threading.CancellationToken option
        OnLine : (ExecutionLine -> Async<unit>) option
    }

let inline executeWithOptionsAsync (options : ExecutionOptions) = async {
    let workingDirectory, fileName, arguments = options.Command |> splitCommand
    let getLocals () = $"workingDirectory: {workingDirectory} / fileName: {fileName} / arguments: {arguments} / {getLocals ()}"
    
    let startInfo = System.Diagnostics.ProcessStartInfo (
        WorkingDirectory = workingDirectory,
        FileName = fileName,
        Arguments = arguments,
        RedirectStandardOutput = true,
        RedirectStandardError = true,
        UseShellExecute = false,
        CreateNoWindow = true
    )

    use proc = new System.Diagnostics.Process (StartInfo = startInfo)
    let output = System.Collections.Concurrent.ConcurrentStack<string> ()

    let inline event error (e: System.Diagnostics.DataReceivedEventArgs) = async {
        if e.Data <> null then
            match options.OnLine with
            | Some onLine ->
                do!
                    onLine
                        {
                            ProcessId = proc.Id
                            Line = e.Data
                            Error = error
                        }
            | None -> ()

            trace
                (if error then Error else Debug)
                (fun () -> $"{if error then 'E' else ' '}{proc.Id}: {e.Data}")
                getLocals

            output.Push
                $"{
                    if error then '['.ToString() else System.String.Empty
                }{
                    e.Data
                }{
                    if error then ']'.ToString() else System.String.Empty
                }"
    }

    proc.OutputDataReceived.Add (event false >> Async.StartImmediate)
    proc.ErrorDataReceived.Add (event true >> Async.StartImmediate)

    trace Debug (fun () -> $"executeAsync") getLocals

    if proc.Start () |> not
    then failwith $"executeAsync / proc.Start() error"

    proc.BeginErrorReadLine ()
    proc.BeginOutputReadLine ()

    let! ct =
        options.CancellationToken
        |> Option.map Async.init
        |> Option.defaultValue Async.CancellationToken

    use reg = ct.Register (fun _ ->
        if not proc.HasExited then proc.Kill ()
    )

    let! exitCode = async {
        try
            do! proc.WaitForExitAsync ct |> Async.AwaitTask
            return proc.ExitCode
        with :? System.Threading.Tasks.TaskCanceledException as ex ->
            trace Warn (fun () -> $"executeAsync / WaitForExitAsync / ex: {ex |> printException}") getLocals
            ex |> printException |> output.Push
            return System.Int32.MinValue
    }

    let output = output |> Seq.rev |> String.concat System.Environment.NewLine

    trace Debug (fun () -> $"executeAsync / exitCode: {exitCode} / output.Length: {output.Length}") getLocals

    return exitCode, output
}

let inline executeAsync command =
    executeWithOptionsAsync
        {
            Command = command
            CancellationToken = None
            OnLine = None
        }

In [ ]:
//// test

let tempFolder = FileSystem.createTempDirectory ()
let path = tempFolder </> "test.txt"

let command = @$"pwsh -c ""Get-Content {path}"""

async {
    let! exitCode, result = executeAsync command
    exitCode |> _equal 1
    result |> _stringContains "not exist"

    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask

    return! executeAsync command
    
}
|> Async.runWithTimeout 10000
|> function
    | Some (exitCode, output) ->
        exitCode |> _equal 0
        output |> _equal "0"

        true
    | _ -> false
|> _equal true

04:43:12 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230729-0443-1294-9416-9513ba0d695e / result: { CreationTime = 2023-07-29 4:43:12 AM
  Exists = true }
04:43:12 #2 [Debug] executeAsync / workingDirectory: . / fileName: pwsh / arguments: -c "Get-Content C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230729-0443-1294-9416-9513ba0d695e\test.txt"
04:43:13 #3 [Error] E20772: Get-Content: Cannot find path 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230729-0443-1294-9416-9513ba0d695e\test.txt' because it does not exist. / workingDirectory: . / fileName: pwsh / arguments: -c "Get-Content C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230729-0443-1294-9416-9513ba0d695e\test.txt"
04:43:14 #4 [Debug] executeAsync / exitCode: 1 / output.Length: 171 / workingDirectory: . / fileName: pwsh / arguments: -c "Get-Content C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230729-0443-1294-9416-9513ba0d695e\test.txt"
1
[Get-Content: Cannot find 

In [ ]:
//// test

let tempFolder = FileSystem.createTempDirectory ()
let path = tempFolder </> "test.txt"

let command = @$"pwsh -c ""Get-Content {path}"""

async {
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    let cts = new System.Threading.CancellationTokenSource ()
    trace Debug (fun () -> $"1") getLocals
    let! result =
        executeWithOptionsAsync
            {
                Command = command
                CancellationToken = Some cts.Token
                OnLine = None
            }
        |> Async.StartChild
    trace Debug (fun () -> $"2") getLocals
    do! Async.Sleep 100
    trace Debug (fun () -> $"3") getLocals
    cts.Cancel ()
    trace Debug (fun () -> $"4") getLocals
    let! exitCode, output = result
    trace Debug (fun () -> $"5") getLocals
    return exitCode, output
}
|> Async.runWithTimeout 10000
|> function
    | Some (exitCode, output) ->
        exitCode |> _equal -2147483648
        output |> _equal "System.Threading.Tasks.TaskCanceledException: A task was canceled."

        true
    | _ -> false
|> _equal true

04:43:14 #8 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230729-0443-1499-9904-9b082af3b6c8 / result: { CreationTime = 2023-07-29 4:43:14 AM
  Exists = true }
04:43:15 #9 [Debug] 1
04:43:15 #10 [Debug] 2
04:43:15 #11 [Debug] executeAsync / workingDirectory: . / fileName: pwsh / arguments: -c "Get-Content C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230729-0443-1499-9904-9b082af3b6c8\test.txt"
04:43:15 #12 [Debug] 3
04:43:15 #13 [Debug] 4
04:43:15 #14 [Warn] executeAsync / WaitForExitAsync / ex: System.Threading.Tasks.TaskCanceledException: A task was canceled. / workingDirectory: . / fileName: pwsh / arguments: -c "Get-Content C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230729-0443-1499-9904-9b082af3b6c8\test.txt"
04:43:15 #15 [Debug] executeAsync / exitCode: -2147483648 / output.Length: 66 / workingDirectory: . / fileName: pwsh / arguments: -c "Get-Content C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230729-0443-1499-9904-9b08